In [88]:
## POlicy iteration and Value iteration

In [89]:
import warnings; warnings.filterwarnings('ignore')

import gym, gym_walk, gym_aima 
import numpy as np 
from pprint import pprint 
from tqdm import tqdm_notebook as tqdm 
from itertools import cycle 
import random 

np.set_printoptions(suppress= True)
random.seed(123); np.random.seed(123)

In [90]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [91]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [92]:
def print_action_value_function(Q, 
                                optimal_Q=None, 
                                action_symbols=('<', '>'), 
                                prec=3, 
                                title='Action-value function:'):
    vf_types=('',) if optimal_Q is None else ('', '*', 'err')
    headers = ['s',] + [' '.join(i) for i in list(itertools.product(vf_types, action_symbols))]
    print(title)
    states = np.arange(len(Q))[..., np.newaxis]
    arr = np.hstack((states, np.round(Q, prec)))
    if not (optimal_Q is None):
        arr = np.hstack((arr, np.round(optimal_Q, prec), np.round(optimal_Q-Q, prec)))
    print(tabulate(arr, headers, tablefmt="fancy_grid"))

In [93]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [94]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [95]:
env = gym.make('SlipperyWalkFive-v0')
P = env.env.P
init_state = env.reset()
goal_state = 6

LEFT, RIGHT = range(2)
pi = lambda s: {
    0:LEFT, 1:LEFT, 2:LEFT, 3:LEFT, 4:LEFT, 5:LEFT, 6:LEFT
}[s]
print_policy(pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=goal_state)*100, 
    mean_return(env, pi)))

Policy:
|           | 01      < | 02      < | 03      < | 04      < | 05      < |           |
Reaches goal 7.00%. Obtains an average undiscounted return of 0.0700.


In [96]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
        V = np.zeros(len(P), dtype=np.float64)
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [97]:
V = policy_evaluation(pi, P)
print_state_value_function(V, P, n_cols=7, prec=5)

State-value function:
|           | 01 0.00275 | 02 0.01099 | 03 0.03571 | 04 0.10989 | 05 0.33242 |           |


### Policy Improvement

In [99]:
def policy_improvement(V, P, gamma = 1.0):
    Q = np.zeros((len(P), len(P[0])), dtype = np.float64)
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]: 
                Q[s][a] += prob * (reward + gamma * V[next_state]* (not done))
    new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis = 1))}[s]
    return new_pi 


In [100]:
improved_pi = policy_improvement(V, P)
print_policy(improved_pi, P, action_symbols=("<", ">"), n_cols = 7)
print("Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.".format(probability_success(env, improved_pi, goal_state = goal_state)*100, mean_return(env, improved_pi)))

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 93.00%. Obtains an average undiscounted return of 0.9300.


In [101]:
# how about we evaluate the improved policy 
improved_V = policy_evaluation(improved_pi, P)
print_state_value_function(improved_V, P, n_cols = 7, prec = 5)

State-value function:
|           | 01 0.66758 | 02 0.89011 | 03 0.96429 | 04 0.98901 | 05 0.99725 |           |


In [104]:
# can we imoroved the imporved policy 
improved_improved_pi = policy_improvement(improved_V, P)
print_policy(improved_improved_pi, P, action_symbols=("<", ">"), n_cols = 7)
print("Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.".format(probability_success(env, improved_improved_pi, goal_state = goal_state)*100, mean_return(env, improved_improved_pi)))


Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 93.00%. Obtains an average undiscounted return of 0.9300.


In [ ]:
def policy_evaluation(pi, P, gamma = 1.0, theta = 1e-10):
    prev_V = np.zeros(len(P), dtype= np.float64)
    while True: 
        V = np.zeros(len(P), dtype = np.float64)
        for s in range(len(P)): 
            for prob, next_state, reward, done in P[s][pi(s)]: 
                V[s] += prob *( reward + gamma * prev_V[next_state]*(not done))
        if np.max(np.abs(prev_V-V)) < theta: 
            break 
        prev_V = V.copy()
    return V 

In [115]:
Q = np.zeros((len(P), len(P[0])))
{s:a for s,a in enumerate(np.argmax(Q, axis = 1))}

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

In [116]:
def policy_improvement(V, P, gamma = 1.0): 
    Q = np.zeros((len(P), len(P[0])), dtype = np.float64)

    for s in range(len(P)): 
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]: 
                Q[s][a] += prob*(reward + gamma*V[next_state] *(not done))
    new_pi = lambda s: {s:a for s,a in enumerate(np.argmax(Q, axis = 1))}[s]

    return new_pi 

### Policy Iteration

In [118]:
def policy_iteration(P, gamma = 1.0, theta = 1e-10):
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]

    while True: 
        old_pi = {s: pi(s) for s in range(len(P))}

        V = policy_evaluation(pi, P, gamma, theta)
        pi = policy_improvement(V, P, gamma)

        if old_pi == {s: pi(s) for s in range(len(P))}:
            break 

    return V, pi     

In [119]:
optimal_V, Optimal_pi = policy_iteration(P)
print("Optimial policy and state-value function (PI):")
print_policy(Optimal_pi, P, action_symbols=("<", ">"), n_cols = 7)
print("Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.".format(probability_success(env, Optimal_pi, goal_state=goal_state)*100, mean_return(env, Optimal_pi))) 
print()
print_state_value_function(optimal_V, P, n_cols = 7, prec = 5)  

Optimial policy and state-value function (PI):
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 93.00%. Obtains an average undiscounted return of 0.9300.

State-value function:
|           | 01 0.66758 | 02 0.89011 | 03 0.96429 | 04 0.98901 | 05 0.99725 |           |


### Frozen Lake MDP and sample policies 

In [147]:
env = gym.make("FrozenLake-v0")
P = env.env.P 
init_state = env.reset()
goal_state = 15

LEFT, DOWN, RIGHT, UP = range(4)
random_pi = lambda s: {
    0:RIGHT, 1:LEFT, 2:DOWN, 3:UP,
    4:LEFT, 5:LEFT, 6:RIGHT, 7:LEFT,
    8:UP, 9:DOWN, 10:UP, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:DOWN, 15:LEFT
}[s]

print_policy(random_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, random_pi, goal_state=goal_state)*100, 
    mean_return(env, random_pi)))

Policy:
| 00      > | 01      < | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      ^ |           |
|           | 13      > | 14      v |           |
Reaches goal 12.00%. Obtains an average undiscounted return of 0.1200.


In [121]:

go_get_pi = lambda s: {
    0:RIGHT, 1:RIGHT, 2:DOWN, 3:LEFT,
    4:DOWN, 5:LEFT, 6:DOWN, 7:LEFT,
    8:RIGHT, 9:RIGHT, 10:DOWN, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}[s]
print_policy(go_get_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, go_get_pi, goal_state=goal_state)*100, 
    mean_return(env, go_get_pi)))

Policy:
| 00      > | 01      > | 02      v | 03      < |
| 04      v |           | 06      v |           |
| 08      > | 09      > | 10      v |           |
|           | 13      > | 14      > |           |
Reaches goal 5.00%. Obtains an average undiscounted return of 0.0500.


In [122]:

careful_pi = lambda s: {
    0:LEFT, 1:UP, 2:UP, 3:UP,
    4:LEFT, 5:LEFT, 6:UP, 7:LEFT,
    8:UP, 9:DOWN, 10:LEFT, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:RIGHT, 15:LEFT
}[s]
print_policy(careful_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, careful_pi, goal_state=goal_state)*100, 
    mean_return(env, careful_pi)))

Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      ^ |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 52.00%. Obtains an average undiscounted return of 0.5200.


In [123]:
V = policy_evaluation(careful_pi, P, gamma=.99)
print_state_value_function(V, P, prec= 4)

State-value function:
| 00 0.4079 | 01 0.3754 | 02 0.3543 | 03 0.3438 |
| 04 0.4203 |           | 06 0.1169 |           |
| 08 0.4454 | 09  0.484 | 10 0.4328 |           |
|           | 13 0.5884 | 14 0.7107 |           |


In [124]:
careful_plus_pi = policy_improvement(V, P, gamma= .99)
print_policy(careful_plus_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, careful_plus_pi, goal_state=goal_state)*100, 
    mean_return(env, careful_plus_pi)))

Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 74.00%. Obtains an average undiscounted return of 0.7400.


In [125]:
new_V = policy_evaluation(careful_plus_pi, P, gamma=0.99)
print_state_value_function(new_V, P, prec=4)

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


In [126]:
print_state_value_function(new_V-V, P, prec= 4)

State-value function:
| 00 0.1341 | 01 0.1234 | 02 0.1164 | 03  0.113 |
| 04 0.1381 |           | 06 0.2414 |           |
| 08 0.1464 | 09 0.1591 | 10 0.1824 |           |
|           | 13 0.1533 | 14 0.1521 |           |


In [127]:
adversarial_pi = lambda s: {
    0:UP, 1:UP, 2:UP, 3:UP,
    4:UP, 5:LEFT, 6:UP, 7:LEFT,
    8:LEFT, 9:LEFT, 10:LEFT, 11:LEFT,
    12:LEFT, 13:LEFT, 14:LEFT, 15:LEFT
}[s]
print_policy(adversarial_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, adversarial_pi, goal_state=goal_state)*100, 
    mean_return(env, adversarial_pi)))

Policy:
| 00      ^ | 01      ^ | 02      ^ | 03      ^ |
| 04      ^ |           | 06      ^ |           |
| 08      < | 09      < | 10      < |           |
|           | 13      < | 14      < |           |
Reaches goal 0.00%. Obtains an average undiscounted return of 0.0000.


In [128]:

V = policy_evaluation(adversarial_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=2)

State-value function:
| 00    0.0 | 01    0.0 | 02    0.0 | 03    0.0 |
| 04    0.0 |           | 06    0.0 |           |
| 08    0.0 | 09    0.0 | 10    0.0 |           |
|           | 13    0.0 | 14    0.0 |           |


In [129]:

i_pi = policy_improvement(V, P, gamma=0.99)
print_policy(i_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, i_pi, goal_state=goal_state)*100, 
    mean_return(env, i_pi)))

Policy:
| 00      < | 01      < | 02      < | 03      < |
| 04      < |           | 06      < |           |
| 08      < | 09      < | 10      < |           |
|           | 13      < | 14      v |           |
Reaches goal 0.00%. Obtains an average undiscounted return of 0.0000.


In [132]:

i_V = policy_evaluation(i_pi, P, gamma=0.99)
print_state_value_function(i_V, P, prec=2)

State-value function:
| 00    0.0 | 01    0.0 | 02   0.04 | 03   0.02 |
| 04    0.0 |           | 06   0.07 |           |
| 08    0.0 | 09    0.0 | 10   0.19 |           |
|           | 13    0.0 | 14    0.5 |           |


In [133]:

ii_pi = policy_improvement(i_V, P, gamma=0.99)
print_policy(ii_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, ii_pi, goal_state=goal_state)*100, 
    mean_return(env, ii_pi)))

Policy:
| 00      < | 01      v | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      < | 09      v | 10      < |           |
|           | 13      v | 14      > |           |
Reaches goal 0.00%. Obtains an average undiscounted return of 0.0000.


In [134]:

ii_V = policy_evaluation(ii_pi, P, gamma=0.99)
print_state_value_function(ii_V, P, prec=2)

State-value function:
| 00    0.0 | 01   0.05 | 02   0.16 | 03   0.15 |
| 04    0.0 |           | 06   0.17 |           |
| 08    0.0 | 09   0.22 | 10   0.35 |           |
|           | 13   0.33 | 14   0.67 |           |


In [135]:

iii_pi = policy_improvement(ii_V, P, gamma=0.99)
print_policy(iii_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, iii_pi, goal_state=goal_state)*100, 
    mean_return(env, iii_pi)))

Policy:
| 00      v | 01      > | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      v | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 20.00%. Obtains an average undiscounted return of 0.2000.


### Value Iteration

In [140]:

def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:
                    Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            break
        V = np.max(Q, axis=1)
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return V, pi

In [153]:

env = gym.make('SlipperyWalkFive-v0')
init_state = env.reset()
goal_state = 6
P = env.env.P

In [154]:
optimal_V, optimal_pi = value_iteration(P)
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<','>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100, 
    mean_return(env, optimal_pi)))
print()
print_state_value_function(optimal_V, P, n_cols=7, prec=5)
# |            | 01   0.668 | 02   0.890 | 03   0.964 | 04   0.989 | 05   0.997 |

Optimal policy and state-value function (PI):
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 93.00%. Obtains an average undiscounted return of 0.9300.

State-value function:
|           | 01 0.66758 | 02 0.89011 | 03 0.96429 | 04 0.98901 | 05 0.99725 |           |
